In [5]:
import pandas_datareader as wb
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [6]:

# 코스피지수 크롤링
#약 3년치 데이터를 크롤링해 옴
start = datetime.datetime(2019, 8, 1)
end = datetime.datetime(2022, 8, 1)

# ^KS11 : 코스피
df_null = wb.DataReader("^KS11","yahoo",start,end)

# 결측치 제거
df = df_null.dropna()

# Close와 Adj Close는 중복되는 columns인것을 확인 함
# Adj Close열을 제거
df.drop(["Adj Close"],axis=1, inplace=True)

# 새로운 칼럼 생성
# (Price : 당일 대비 다음날 주가가 상승했으면 1, 하락했으면 0 표시)
df['Price'] = 0
for i in range(len(df)-1):
    if df['Close'][i] < df['Close'][i+1]:
        df['Price'][i] = 1
    else:
        df['Price'][i] = 0

# columns명을 알기 쉽게 한글로 변경
df.columns = ["최고가", "최저가" , "시작가", "종가", "거래량" , "등락"]

# 파일 저장
df.to_csv('kospi_주가데이터.csv',encoding='utf-8-sig')
df


/var/folders/3h/jprthrkx73n0d9pl0mgr0kqm0000gn/T/ipykernel_82322/2642073490.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 0
/var/folders/3h/jprthrkx73n0d9pl0mgr0kqm0000gn/T/ipykernel_82322/2642073490.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 1


,최고가,최저가,시작가,종가,거래량,등락
Date,,,,,,
2019-08-01,2032.229980,2009.329956,2015.089966,2017.339966,461700,0
2019-08-02,2005.310059,1987.119995,1995.310059,1998.130005,438500,0
2019-08-05,1987.640015,1945.390015,1985.930054,1946.979980,639600,0
2019-08-06,1948.530029,1891.810059,1900.359985,1917.500000,739600,0
2019-08-07,1929.290039,1901.609985,1925.329956,1909.709961,760800,1
...,...,...,...,...,...,...
2022-07-26,2415.389893,2395.169922,2397.270020,2412.959961,415100,1
2022-07-27,2415.530029,2396.189941,2412.520020,2415.530029,333400,1
2022-07-28,2443.429932,2429.070068,2437.570068,2435.270020,490900,1


In [7]:
# 등락 값이 "0"인 데이터와 "1"인 데이터를 나눠서 리스트화
# 여기서 리스트화 하는 이유는 다음날 주가가 하락한 경우의 뉴스 타이틀과 상승한 뉴스타이틀을 크롤링해 분류하기 위함
price_data = pd.read_csv('kospi_주가데이터.csv')

#주가가 하락한 경우의 Date 데이터 리스트 date_0
df_0 = price_data[price_data['등락']==0]['Date']
date_0 = []
for i in range(0,len(df_0)):
    date_0.append(str(df_0.tolist()[i])[:10].replace('-',''))

#주가가 상승한 경우의 Date 데이터 리스트 date_1
df_1 = price_data[price_data['등락']==1]['Date']
date_1 = []
for i in range(0,len(df_1)):
    date_1.append(str(df_1.tolist()[i])[:10].replace('-',''))

date_1

['20190807',
 '20190808',
 '20190809',
 '20190813',
 '20190816',
 '20190819',
 '20190820',
 '20190826',
 '20190827',
 '20190829',
 '20190830',
 '20190903',
 '20190904',
 '20190905',
 '20190906',
 '20190909',
 '20190910',
 '20190911',
 '20190916',
 '20190917',
 '20190918',
 '20190919',
 '20190920',
 '20190923',
 '20190925',
 '20190927',
 '20190930',
 '20191004',
 '20191007',
 '20191010',
 '20191011',
 '20191014',
 '20191015',
 '20191018',
 '20191021',
 '20191023',
 '20191024',
 '20191025',
 '20191030',
 '20191031',
 '20191101',
 '20191104',
 '20191105',
 '20191106',
 '20191111',
 '20191113',
 '20191114',
 '20191121',
 '20191122',
 '20191126',
 '20191129',
 '20191205',
 '20191206',
 '20191209',
 '20191210',
 '20191211',
 '20191212',
 '20191216',
 '20191218',
 '20191219',
 '20191224',
 '20191226',
 '20200102',
 '20200106',
 '20200108',
 '20200109',
 '20200110',
 '20200113',
 '20200115',
 '20200116',
 '20200117',
 '20200121',
 '20200128',
 '20200203',
 '20200204',
 '20200205',
 '20200210',

In [8]:
#뉴스타이틀 크롤링을 위한 모듈 import
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse


In [9]:
# 팍스넷 크롤링 함수 paxnet_news_title
result_list = []
error_cnt = 0

def paxnet_news_title(dates):
    base_url = 'http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo={}&genDate={}&objId=N4667'
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
    }
    
    for date in dates:
        for page in range(1, 3):
            url = base_url.format(page, date)
            res = requests.get(url, headers=headers)
            if res.status_code == 200:
                soup = BeautifulSoup(res.text)
                title_list = soup.select('ul.thumb-list li')
                for title in title_list:
                    try:
                        news_title = title.select_one('dl.text > dt').text.strip()
                        #"[문자]" 가 붙은 제거
                        #find 함수는 문자열안에 지정한 문자가 있을경우 그 위치(index)를 int로 반환해준다.
                        #지정한 문자를 찾지 못할 경우 -1 을 반환 
                        find_1=news_title.find("[")
                        find_2=news_title.find("]")

                        # find함수의 결과가 -1가 아닌 경우(문자열안에 "[문자]"가 있다.)
                        # 그 위치를 찾아 제거하고 result_list에 추가
                        if find_1 != -1:
                            slice_news_title=news_title[find_1:find_2+1]
                            result_list.append([news_title.strip(slice_news_title).strip()])
                        #find함수의 결과가 -1 일경우(문자열안에 "[문자]"가 없다.)
                        #바로 result_list에 추가
                        elif find_1 == -1:
                            result_list.append([news_title])
                    except:
                        error_cnt += 1
paxnet_news_title(date_0)
title_df_0 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_0['주가변동'] = 0
result_list = []

paxnet_news_title(date_1)
title_df_1 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_1['주가변동'] = 1
result_list = []



#팍스넷 크롤링 데이터 합쳐 csv파일로 만들기
title_df = pd.concat([title_df_0, title_df_1])
#중복 데이터 삭제 
del_title_df = title_df.drop_duplicates(['뉴스제목'])
#데이터프레임 저장 
del_title_df.to_csv('팍스넷_뉴스타이틀.csv', index=False, encoding='utf-8-sig')



In [10]:
# 네이버 크롤링 함수 naver_news_title
result_list = []
error_cnt = 0
def naver_news_title(dates):
    base_url = 'https://finance.naver.com/news/mainnews.naver?date={}&page={}'
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
    }
    for date in dates:
        for page in range(1, 3):
            url = base_url.format(date,page)
            res = requests.get(url, headers=headers)
            if res.status_code == 200:
                soup = BeautifulSoup(res.text)
                title_list = soup.select('#contentarea_left > div.mainNewsList > ul > li > dl')
                for title in title_list:
                    try:
                        news_title = title.select_one('.articleSubject').text.strip()
                        find_1=news_title.find("[")
                        find_2=news_title.find("]")
                        if find_1 != -1:
                            slice_news_title=news_title[find_1:find_2+1]
                            news_title.strip(slice_news_title)
                            result_list.append([news_title.strip(slice_news_title).strip()])
                        elif find_1 == -1:
                            result_list.append([news_title])
                    except:
                        error_cnt += 1

naver_news_title(date_0)
title_df_2 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_2['주가변동'] = 0
result_list = []

naver_news_title(date_1)
title_df_3 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_3['주가변동'] = 1
result_list = []

#네이버 크롤링 데이터 합쳐 csv파일로 만들기
title_df2 = pd.concat([title_df_2, title_df_3])
#중복 데이터 삭제 
del_title_df2 = title_df2.drop_duplicates(['뉴스제목'])
title_df2.to_csv('네이버_뉴스타이틀.csv', index=False, encoding='utf-8-sig')
title_df2


,뉴스제목,주가변동
0,초반 상승세…기업 실적에 다시 관심,0
1,日 화이트리스트 제외?…코스피 2000선 무너지나,0
2,美 FOMC에 日 수출규제까지…원화가치 급락,0
3,美 금리인하에도 시장은 '실망',0
4,"코스피, '日 화이트리스트 제외' 앞두고 하락…7개월 만에 최저치",0
...,...,...
12650,美증시 상승에 코스피 강세…2450선 출발,1
12651,"美경기 부진, 긴축 속도조절론↑…환율, 1290원대 이틀째 하락 예상",1
12652,"GDP '연속 마이너스'에도 美증시↑...""거의 다 왔다""",1
12653,미국 두 분기 연속 역성장 반긴 뉴욕증시…전국 찜통더위 계속,1


In [78]:
# 다음 크롤링 함수 daum_news_title
result_list = []
error_cnt = 0
def daum_news_title(dates):
    base_url = "https://news.daum.net/breakingnews/economic/stock?page={1}&regDate={0}"
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
    }
    for date in dates:
        for page in range(1, 3):
            url = base_url.format(date, page)
            res = requests.get(url, headers=headers)
            if res.status_code == 200:
                soup = BeautifulSoup(res.text)
                title_list = soup.select('ul.list_news2.list_allnews > li')
                for title in title_list:
                    try:
                        news_title = title.select_one('.link_txt').text.strip()
                        find_1=news_title.find("[")
                        find_2=news_title.find("]")
                        if find_1 != -1:
                            slice_news_title=news_title[find_1:find_2+1]
                            news_title.strip(slice_news_title)
                            result_list.append([news_title.strip(slice_news_title).strip()])
                        elif find_1 == -1:
                            result_list.append([news_title])
                    except:
                        error_cnt += 1

daum_news_title(date_0)
title_df_4 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_4['주가변동'] = 0
result_list = []

daum_news_title(date_1)
title_df_5 = pd.DataFrame(result_list, columns=['뉴스제목'])
title_df_5['주가변동'] = 1
result_list = []

#다음 크롤링 데이터 합쳐 csv파일로 만들기
title_df3 = pd.concat([title_df_4, title_df_5])
#중복 데이터 삭제 
del_title_df3 = title_df3.drop_duplicates(['뉴스제목'])
del_title_df3.to_csv('다음_뉴스타이틀.csv', index=False, encoding='utf-8-sig')
del_title_df3


,뉴스제목,주가변동
0,슈프리마아이디 및 한국바이오젠 코스닥 상장,0
1,"이주열 한은 총재 ""미국 연준, 예상보다 덜 완화적""",0
2,"뉴욕증시, 연준 금리 경로 불확실성 속 상승 출발",0
3,美 실업수당 청구 21만5000건..예상치 상회,0
4,"英 중앙은행, 올해 성장률 1.3%로 하향조정..기준금리는 동결",0
...,...,...
12295,"29일, 거래소 기관 순매수상위에 운수장비 업종 4종목",1
12296,"29일, 거래소 외국인 순매수상위에 운수장비 업종 5종목",1
12297,"기다리던 공매도 대책 나왔는데..개미 반응은 싸늘, 왜?",1
12298,"효성, 2분기 영업익 356억원..전년대비 83.6% 감소",1


In [11]:
#뉴스데이터 하나로 합치기 
naver_data = pd.read_csv('네이버_뉴스타이틀.csv')
paxnet_data = pd.read_csv('팍스넷_뉴스타이틀.csv')
all_title = pd.concat([naver_data, paxnet_data])
all_title.to_csv('paxnet_naver.csv')